# Pipeline para Generación de Agonistas de PD-L1

Este notebook implementa un pipeline completo para generar agonistas de PD-L1 usando:
- **RF Diffusion**: Generación de estructuras proteicas
- **Protein MPNN**: Diseño de secuencias
- **AlphaFold3**: Validación de estructuras

## 1. Configuración e Importaciones

In [1]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np

# Añadir src al path
project_root = Path.cwd().parent
sys.path.append(str(project_root))

from src.rf_diffusion_generator import RFDiffusionGenerator, generate_multiple_structures
from src.protein_mpnn_generator import ProteinMPNNGenerator, generate_sequences_for_multiple_structures
from src.alphafold3_predictor import AlphaFold3Predictor, validate_mpnn_sequences_with_alphafold3

In [2]:
# Configuración de rutas
data_dir = project_root / "data"
raw_data_dir = data_dir / "raw"
processed_data_dir = data_dir / "processed"

# Crear directorios necesarios
processed_data_dir.mkdir(parents=True, exist_ok=True)

# Rutas específicas para PD-L1
pdl1_data_dir = processed_data_dir / "pdl1_agonists"
pdl1_data_dir.mkdir(parents=True, exist_ok=True)

## 2. Paso 1: Generación de Estructuras con RF Diffusion

**Nota**: Necesitas una API key de NVIDIA Build. Obtén una en: https://build.nvidia.com/

El archivo PDB se descargo desde https://www.rcsb.org/structure/7SJQ

Revisión del las cadenas del PDB

In [3]:
from Bio.PDB import PDBParser

# Si tienes una estructura de PD-L1 como template
pdl1_pdb = raw_data_dir / "PDL1_structure.pdb"  # Ajusta la ruta según tu archivo

parser = PDBParser(QUIET=True)
structure = parser.get_structure("PDL1", pdl1_pdb)

for model in structure:
    for chain in model:
        residues = list(chain.get_residues())
        first_res = residues[0].get_id()[1]
        last_res = residues[-1].get_id()[1]
        print(f"Chain {chain.id}: residues {first_res} - {last_res}")


Chain A: residues 18 - 374
Chain D: residues 2 - 103


Arreglamos el archivo PDB mediante structure normalization / preparation

In [4]:
from Bio.PDB import PDBParser, PDBIO

parser = PDBParser(QUIET=True)
structure = parser.get_structure("PDL1", pdl1_pdb)

# Tomar solo chain A
model = structure[0]
chain_a = model['A']

# Crear nueva estructura
from Bio.PDB.Structure import Structure
from Bio.PDB.Model import Model
from Bio.PDB.Chain import Chain

new_structure = Structure("PDL1_clean")
new_model = Model(0)
new_chain = Chain("A")

new_res_id = 1
for res in chain_a:
    if res.id[0] == " ":  # solo residuos estándar
        new_res = res.copy()
        new_res.id = (" ", new_res_id, " ")
        new_chain.add(new_res)
        new_res_id += 1

new_model.add(new_chain)
new_structure.add(new_model)

# Guardar
io = PDBIO()
io.set_structure(new_structure)
clean_pdb = pdl1_pdb.parent / "PDL1_clean.pdb"
io.save(str(clean_pdb))


print(f"PDB limpio guardado en: {clean_pdb}")


PDB limpio guardado en: e:\Documentos\Maestria\Proyecto Integrador\Codigo\glp-1_drug_discovery\data\raw\PDL1_clean.pdb


In [5]:
from Bio.PDB import PDBParser

# Si tienes una estructura de PD-L1 como template
pdl1_pdb = raw_data_dir / "PDL1_clean.pdb"  # Ajusta la ruta según tu archivo

parser = PDBParser(QUIET=True)
structure = parser.get_structure("PDL1", pdl1_pdb)

for model in structure:
    for chain in model:
        residues = list(chain.get_residues())
        first_res = residues[0].get_id()[1]
        last_res = residues[-1].get_id()[1]
        print(f"Chain {chain.id}: residues {first_res} - {last_res}")

Chain A: residues 1 - 117


In [6]:
from Bio.PDB import PDBParser

# Si tienes una estructura de PD-L1 como template
pdl1_pdb = raw_data_dir / "PDL1_clean.pdb"  # Ajusta la ruta según tu archivo

parser = PDBParser(QUIET=True)
structure = parser.get_structure("PDL1", pdl1_pdb)

res_ids = {
    f"A{res.get_id()[1]}"
    for res in structure[0]["A"]
}

print(res_ids)


{'A97', 'A89', 'A29', 'A110', 'A79', 'A9', 'A59', 'A98', 'A23', 'A31', 'A58', 'A13', 'A8', 'A74', 'A49', 'A48', 'A114', 'A18', 'A73', 'A67', 'A78', 'A60', 'A70', 'A4', 'A52', 'A17', 'A85', 'A5', 'A94', 'A16', 'A21', 'A34', 'A105', 'A83', 'A35', 'A45', 'A71', 'A47', 'A100', 'A64', 'A72', 'A57', 'A61', 'A26', 'A116', 'A46', 'A66', 'A55', 'A11', 'A102', 'A107', 'A87', 'A101', 'A24', 'A75', 'A111', 'A14', 'A76', 'A80', 'A27', 'A42', 'A39', 'A10', 'A81', 'A62', 'A113', 'A1', 'A12', 'A112', 'A92', 'A6', 'A33', 'A54', 'A93', 'A99', 'A36', 'A2', 'A3', 'A63', 'A38', 'A65', 'A117', 'A44', 'A96', 'A30', 'A56', 'A28', 'A106', 'A88', 'A40', 'A43', 'A7', 'A82', 'A109', 'A15', 'A32', 'A103', 'A95', 'A69', 'A104', 'A108', 'A115', 'A91', 'A37', 'A41', 'A77', 'A86', 'A22', 'A68', 'A25', 'A53', 'A84', 'A51', 'A19', 'A50', 'A20', 'A90'}


In [7]:
# Configurar RF Diffusion
# IMPORTANTE: Reemplaza con tu API key real
NVIDIA_API_KEY = "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"  # Obtén una en https://build.nvidia.com/

rf_diffusion = RFDiffusionGenerator(api_key=NVIDIA_API_KEY)

# Si tienes una estructura de PD-L1 como template
pdl1_pdb = raw_data_dir / "PDL1_clean.pdb"  # Ajusta la ruta según tu archivo

# Generar estructuras de binders para PD-L1
if pdl1_pdb.exists():
    print(f"Usando template de PD-L1: {pdl1_pdb}")
    
    # Generar múltiples estructuras de binders
    #binder_lengths = [30, 40, 50, 60]  # Diferentes longitudes a probar
    binder_lengths = [30]  # Diferentes longitudes a probar
    
    results = []
    for length in binder_lengths:
        print(f"\nGenerando binder de longitud {length}...")
        try:
            #hotspots = [{"chain_id": "A", "residue_index": i} for i in range(50, 121)]
            #hotspots = ",".join([f"A:{i}" for i in range(50, 121)])
            #hotspots = list(range(50, 121))
            #hotspots = [f"A{i}" for i in range(50, 121)]
            hotspots = ["A54", "A56", "A60", "A62", "A66", "A68", "A70"]
            result = rf_diffusion.generate_pdl1_binder(
                pdl1_pdb=str(pdl1_pdb),
                binder_length=length,
                hotspot_residues=None,
                output_dir=pdl1_data_dir / "rfdiffusion_structures"
            )
            results.append(result)
        except Exception as e:
            print(f"Error generando binder de longitud {length}: {e}")
else:
    print(f"ADVERTENCIA: Archivo PDB de PD-L1 no encontrado en {pdl1_pdb}")
    print("Generando estructuras sin template...")
    
    # Generar estructuras de novo
    contigs_list = ["A1-50", "A1-60", "A1-70"]  # Diferentes tamaños
    results = generate_multiple_structures(
        generator=rf_diffusion,
        contigs_list=contigs_list,
        output_dir=pdl1_data_dir / "rfdiffusion_structures"
    )

Usando template de PD-L1: e:\Documentos\Maestria\Proyecto Integrador\Codigo\glp-1_drug_discovery\data\raw\PDL1_clean.pdb

Generando binder de longitud 30...


Convertir el resultado JSON a PDB

In [8]:
import json
from pathlib import Path

# Carpeta donde están los JSON generados por RF Diffusion
json_dir = pdl1_data_dir / "rfdiffusion_structures"  # Ajusta la ruta según tu proyecto
pdb_dir = json_dir / "pdb_files"  # Carpeta de salida para los PDB
pdb_dir.mkdir(exist_ok=True)

# Recorrer todos los JSON
json_files = list(json_dir.glob("*.json"))
print(f"Encontrados {len(json_files)} archivos JSON")
    
for json_file in json_files:
    with open(json_file, "r") as f:
        data = json.load(f)
    
    # Extraer el PDB
    pdb_str = data.get("output_pdb")
    if not pdb_str:
        print(f"Advertencia: no se encontró 'output_pdb' en {json_file}")
        continue
    
    # Guardar como archivo .pdb
    pdb_file = pdb_dir / f"{json_file.stem}.pdb"
    with open(pdb_file, "w") as f:
        f.write(pdb_str)
    
    print(f"PDB generado: {pdb_file}")

print("Todos los archivos PDB han sido generados")


Encontrados 12 archivos JSON
PDB generado: e:\Documentos\Maestria\Proyecto Integrador\Codigo\glp-1_drug_discovery\data\processed\pdl1_agonists\rfdiffusion_structures\pdb_files\rfdiffusion_1769482285.pdb
PDB generado: e:\Documentos\Maestria\Proyecto Integrador\Codigo\glp-1_drug_discovery\data\processed\pdl1_agonists\rfdiffusion_structures\pdb_files\rfdiffusion_1769483697.pdb
PDB generado: e:\Documentos\Maestria\Proyecto Integrador\Codigo\glp-1_drug_discovery\data\processed\pdl1_agonists\rfdiffusion_structures\pdb_files\rfdiffusion_1769486029.pdb
PDB generado: e:\Documentos\Maestria\Proyecto Integrador\Codigo\glp-1_drug_discovery\data\processed\pdl1_agonists\rfdiffusion_structures\pdb_files\rfdiffusion_1769486129.pdb
PDB generado: e:\Documentos\Maestria\Proyecto Integrador\Codigo\glp-1_drug_discovery\data\processed\pdl1_agonists\rfdiffusion_structures\pdb_files\rfdiffusion_1769486397.pdb
PDB generado: e:\Documentos\Maestria\Proyecto Integrador\Codigo\glp-1_drug_discovery\data\processed\p

## 3. Paso 2: Diseño de Secuencias con Protein MPNN

Para cada estructura generada por RF Diffusion, diseñamos secuencias que se plieguen a esa estructura.

De momento esto es un TODO. Por ahora se esta usando protein desde el CLI. La libreria CLI se instalo con pip install proteinmpnn pero el plan es bajar el repo y usar el codigo del repo en su lugar

Se clono el repo https://github.com/dauparas/ProteinMPNN

Luego se instalo con pip install -e .

In [9]:
import shutil
print(shutil.which("proteinmpnn"))


C:\Users\Nitro\AppData\Roaming\Python\Python312\Scripts\proteinmpnn.EXE


In [10]:
# Configurar Protein MPNN
# Opción 1: Usar el paquete pip (recomendado)
mpnn_generator = ProteinMPNNGenerator(
    mpnn_path=r"C:\Users\Nitro\AppData\Roaming\Python\Python312\Scripts\proteinmpnn.EXE"
)


# Opción 2: Usar CLI local
# mpnn_generator = ProteinMPNNGenerator(mpnn_path="/ruta/a/protein_mpnn.py")

# Opción 3: Usar API de Levitate Bio
# mpnn_generator = ProteinMPNNGenerator(use_api=True, api_key="TU_API_KEY")

# Obtener archivos PDB generados por RF Diffusion
rfdiffusion_dir = pdl1_data_dir / "rfdiffusion_structures/pdb_files"
pdb_files = list(rfdiffusion_dir.glob("*.pdb"))

if pdb_files:
    print(f"Encontrados {len(pdb_files)} archivos PDB")
    
    # Generar secuencias para cada estructura
    all_sequences_df = generate_sequences_for_multiple_structures(
        generator=mpnn_generator,
        pdb_files=[str(pdb) for pdb in pdb_files],
        num_designs_per_structure=20,  # 20 secuencias por estructura
        output_dir=pdl1_data_dir / "mpnn_sequences"
    )
    
    print(f"\nGeneradas {len(all_sequences_df)} secuencias únicas")
    print(all_sequences_df.head())
else:
    print("No se encontraron archivos PDB de RF Diffusion")

Encontrados 12 archivos PDB
Generando secuencias para e:\Documentos\Maestria\Proyecto Integrador\Codigo\glp-1_drug_discovery\data\processed\pdl1_agonists\rfdiffusion_structures\pdb_files\rfdiffusion_1769482285.pdb...
❌ Error ejecutando ProteinMPNN
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Nitro\AppData\Roaming\Python\Python312\Scripts\proteinmpnn.EXE\__main__.py", line 7, in <module>
  File "C:\Users\Nitro\AppData\Roaming\Python\Python312\site-packages\proteinmpnn\protein_mpnn_run.py", line 374, in main
    with open(ali_file, 'w') as f:
         ^^^^^^^^^^^^^^^^^^^
OSError: [Errno 22] Invalid argument: 'e:\\Documentos\\Maestria\\Proyecto Integrador\\Codigo\\glp-1_drug_discovery\\data\\processed\\pdl1_agonists\\mpnn_sequences\\mpnn_output//seqs/e:\\Documentos\\Maestria\\Proyecto Integrador\\Codigo\\glp-1_drug_discovery\\data\\processed\\pdl1_agonists\\mpnn_sequences\\mpnn_

## 4. Paso 3: Validación con AlphaFold3

Validamos que las secuencias generadas por Protein MPNN se plieguen correctamente usando AlphaFold3.

In [ ]:
# Configurar AlphaFold3
# IMPORTANTE: Necesitas tener AlphaFold3 instalado y acceso a los parámetros del modelo
# Ver: https://github.com/google-deepmind/alphafold3

alphafold3_path = "/ruta/a/alphafold3/run_alphafold3.py"  # Ajusta según tu instalación
model_params_dir = "/ruta/a/alphafold3/params"  # Directorio con parámetros del modelo

af3_predictor = AlphaFold3Predictor(
    alphafold3_path=alphafold3_path,
    model_params_dir=model_params_dir
)

# Validar secuencias generadas por Protein MPNN
if 'all_sequences_df' in locals() and not all_sequences_df.empty:
    print("Validando secuencias con AlphaFold3...")
    
    validation_df = validate_mpnn_sequences_with_alphafold3(
        mpnn_sequences_df=all_sequences_df,
        predictor=af3_predictor,
        output_dir=pdl1_data_dir / "alphafold3_validation"
    )
    
    print(f"\nValidación completada para {len(validation_df)} secuencias")
    print(validation_df.head())
    
    # Filtrar secuencias con predicción exitosa
    successful_predictions = validation_df[validation_df['status'] == 'success']
    print(f"\nSecuencias con predicción exitosa: {len(successful_predictions)}")
else:
    print("No hay secuencias para validar")

## 5. Análisis y Filtrado de Candidatos

Filtramos los candidatos basándonos en:
- Validación exitosa con AlphaFold3
- Propiedades fisicoquímicas
- Predicción de afinidad con PD-L1

In [ ]:
# Cargar utilidades del proyecto existente
from src.ifeature_process import calcular_descriptores_ifeature
from src.PeptideBert_predict import predict_peptidebert

# Filtrar candidatos exitosos
if 'successful_predictions' in locals() and not successful_predictions.empty:
    print(f"Analizando {len(successful_predictions)} candidatos...")
    
    # Calcular descriptores moleculares
    print("\nCalculando descriptores moleculares...")
    
    # Preparar DataFrame para iFeature
    sequences_df = successful_predictions[['sequence_id', 'sequence']].copy()
    sequences_df.columns = ['ID', 'sequence']
    
    # Calcular descriptores (esto requiere archivos FASTA)
    # Primero guardar como FASTA
    from src.bio_utils import save_df_as_fasta
    
    fasta_file = pdl1_data_dir / "candidate_sequences.fasta"
    save_df_as_fasta(
        dataframe=sequences_df,
        id_col='ID',
        seq_col='sequence',
        output_file=fasta_file
    )
    
    # Calcular descriptores
    descriptors_df = calcular_descriptores_ifeature(
        directorio_temporal=pdl1_data_dir / "temp",
        dataframe=sequences_df,
        sequence_col='sequence',
        id_col='ID'
    )
    
    # Evaluar propiedades farmacológicas con PeptideBERT
    print("\nEvaluando propiedades farmacológicas...")
    
    # Nota: Ajusta la ruta al modelo PeptideBERT según tu instalación
    peptidebert_path = project_root / "models" / "peptideBert"
    
    if peptidebert_path.exists():
        properties_df = predict_peptidebert(
            model_directory_path=str(peptidebert_path),
            input_dataframe=sequences_df,
            sequence_col='sequence',
            feats=['hemo', 'sol', 'nf']  # Hemólisis, solubilidad, no adherencia
        )
        
        # Combinar todos los resultados
        final_df = successful_predictions.merge(
            descriptors_df, left_on='sequence_id', right_on='ID', how='left'
        ).merge(
            properties_df, left_on='sequence_id', right_on='ID', how='left'
        )
        
        # Guardar resultados finales
        output_file = pdl1_data_dir / "pdl1_agonist_candidates.csv"
        final_df.to_csv(output_file, index=False)
        
        print(f"\nResultados guardados en: {output_file}")
        print(f"Total de candidatos: {len(final_df)}")
        
        # Mostrar mejores candidatos (ejemplo: alta solubilidad, baja hemólisis)
        if 'sol' in final_df.columns and 'hemo' in final_df.columns:
            best_candidates = final_df[
                (final_df['sol'] > 0.7) &  # Alta solubilidad
                (final_df['hemo'] < 0.1)   # Baja hemólisis
            ].sort_values('sol', ascending=False)
            
            print(f"\nMejores candidatos (alta solubilidad, baja hemólisis): {len(best_candidates)}")
            print(best_candidates[['sequence_id', 'sequence', 'sol', 'hemo']].head(10))
    else:
        print(f"Modelo PeptideBERT no encontrado en {peptidebert_path}")
else:
    print("No hay candidatos exitosos para analizar")

## 6. Próximos Pasos

1. **Docking Molecular**: Usar herramientas como AutoDock Vina para predecir afinidad con PD-L1
2. **Análisis de Estabilidad**: Evaluar estabilidad térmica y enzimática
3. **Síntesis y Validación Experimental**: Seleccionar los mejores candidatos para síntesis